In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [3]:
df = pd.read_csv('C:\\Users\\SMRUTHI DESAI\\Desktop\\E-Commerce Recommendation System\\Womens Clothing E-Commerce Reviews.csv')
df['Review Text'] = df['Review Text'].fillna('')
df.head()


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Review Text'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend_content(product_id, top_n=5):
    idx = df.index[df['Clothing ID'] == product_id][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    product_indices = [i[0] for i in sim_scores]
    return df[['Clothing ID', 'Title', 'Review Text']].iloc[product_indices]


In [5]:
# Create user-item matrix
user_item_matrix = df.pivot_table(index='Age', columns='Clothing ID', values='Rating').fillna(0)

# Compute similarity between users
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [6]:
def recommend_collab(user_id, top_n=5):
    if user_id not in user_item_matrix.index:
        return "User not found"
    
    # Get similar users
    sim_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]
    
    # Weighted ratings
    weighted_ratings = pd.Series(0, index=user_item_matrix.columns)
    sim_sum = 0
    for sim_user, sim_score in sim_users.items():
        weighted_ratings += user_item_matrix.loc[sim_user] * sim_score
        sim_sum += sim_score
    weighted_ratings /= sim_sum
    
    # Remove items already rated by the user
    user_rated_items = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index
    weighted_ratings = weighted_ratings.drop(user_rated_items)
    
    # Return top N recommended products
    top_products = weighted_ratings.sort_values(ascending=False).head(top_n).index
    return df[df['Clothing ID'].isin(top_products)][['Clothing ID', 'Title', 'Review Text']]


In [7]:
user_id = int(input("Enter Age (as user_id): "))
print("Top recommendations (Collaborative):")
print(recommend_collab(user_id))


Top recommendations (Collaborative):
       Clothing ID                                         Title  \
142           1126                                           NaN   
160           1126                               So sad not mine   
164           1126                                    So pretty!   
233           1030  Must have, right on trend, but still classic   
248           1030                                           NaN   
...            ...                                           ...   
22160         1074               Flattering for a summer wedding   
22507          903          Business in front; party in the back   
22754         1030                                           NaN   
22858          903             Flattering and lightweight duster   
22929          903               Love this lightweight cardigan!   

                                             Review Text  
142                                                       
160    Love everything about